In [1]:
sentences1 = ["私は公園で遊びます。", "彼女は本を読みます。", "彼はテニスをします。", "私たちは映画を見に行きます。", "私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。", "私は公園で楽しみます。", "私は公園で転びます。", "私は公園で騒ぎます", "私は公園で歩きます。"]


In [2]:
sentences1 = ["私は公園で遊びます。 彼女は本を読みます。", "彼はテニスをします。 私たちは映画を見に行きます。", "私は公園で騒ぎます。 私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。 私は公園で楽しみます。", "私は公園で転びます。 私は公園で騒ぎます", "私は公園で転びます。 私は公園で歩きます。"]

- pre-trained BERT
- word2vec
- doc2vec
- sentence BERT

In [3]:
# pre-trained BERT

# import numpy as np
# import torch
# from transformers import BertJapaneseTokenizer, BertModel

# BERT_MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# tokenizer = BertJapaneseTokenizer.from_pretrained(BERT_MODEL_NAME)
# model_bert = BertModel.from_pretrained(BERT_MODEL_NAME)

# def sentence_to_vector_bert(model_bert, tokenizer, sentence):
#     tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
#     input = torch.tensor(tokens).unsqueeze(0)
#     with torch.no_grad():
#         outputs = model_bert(input, output_hidden_states=True)
#         last_hidden_state = outputs[0][:, 0, :]
#         averaged_hidden_state = last_hidden_state.mean(
#             dim=0).unsqueeze(0).reshape(-1)
#     return averaged_hidden_state


In [4]:
# sentence_to_vector_bert(model_bert, tokenizer, sentences1[0])

In [5]:
# word2vec

import re
import csv
import numpy as np
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/


In [6]:
def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger(
      '-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')

  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  print(f"sentence1_words: {sentence1_words}")

  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  print(f"sentence2_words: {sentence2_words}")

  if not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    # Compute word embeddings for each sentence
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    # print(f"sentence1_embedding: {sentence1_embedding}")
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    # print(f"sentence2_embedding: {sentence2_embedding}")
    if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
        similarity = 0
    else:
      similarity = cosine_similarity(
          [sentence1_embedding], [sentence2_embedding])[0][0]

  print(f"similarity: {similarity}")

  return similarity


def calc_average_similarity_word2vec(sentences, key):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)

    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["average_similarity"] = np.mean(similarities)
    outputs["size"] = len(list(np.sort(np.array(similarities))))
    outputs["width_q"] = outputs["quantile_3"] - outputs["quantile_1"]

    if key == "quantile_3":
        return outputs["quantile_3"]
    elif key == "average":
        return outputs["average_similarity"]
    elif key == "size":
        return outputs["size"]
    elif key == "width":
        return outputs["width_q"]


In [7]:
sentences1 = ["うす", "うっす"]


In [8]:
calc_average_similarity_word2vec(sentences1, "quantile_3")


sentence1_words: ['うす']
sentence2_words: ['うっす']
similarity: 0


/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.0

In [9]:
calc_average_similarity_word2vec(sentences2, "quantile_3")

sentence1_words: ['私', 'は', '公園', 'で', '遊び', 'ます', '。', '私', 'は', '公園', 'で', '楽しみ', 'ます', '。']
sentence2_words: ['私', 'は', '公園', 'で', '転び', 'ます', '。', '私', 'は', '公園', 'で', '騒ぎ', 'ます']
similarity: 0.971692681312561
sentence1_words: ['私', 'は', '公園', 'で', '遊び', 'ます', '。', '私', 'は', '公園', 'で', '楽しみ', 'ます', '。']
sentence2_words: ['私', 'は', '公園', 'で', '転び', 'ます', '。', '私', 'は', '公園', 'で', '歩き', 'ます', '。']
similarity: 0.9858956933021545
sentence1_words: ['私', 'は', '公園', 'で', '転び', 'ます', '。', '私', 'は', '公園', 'で', '騒ぎ', 'ます']
sentence2_words: ['私', 'は', '公園', 'で', '転び', 'ます', '。', '私', 'は', '公園', 'で', '歩き', 'ます', '。']
similarity: 0.9809364080429077


0.9834160506725311